<a href="https://colab.research.google.com/github/Hyeonwid/Lyric_DLProject/blob/main/CarlDavidLyricPrj2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
!pip install datasets
from datasets import load_dataset

# TODO: change to GPT2 Tokenization Method 
!pip3 install transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TFGPT2LMHeadModel, AutoConfig, TextStreamer #, AutoModelForCausalLM, AutoTokenizer
base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2') #AutoTokenizer.from_pretrained("gpt2")
base_model = GPT2LMHeadModel.from_pretrained('gpt2') #AutoModelForCausalLM.from_pretrained("gpt2")
# from transformers import BertTokenizer
# from tokenizers import BertWordPieceTokenizer

# from keras.utils import to_categorical
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.text import text_to_word_sequence
# from keras.preprocessing.sequence import pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Early on experiment with bert tokenizer and fine tuning it

# bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# tokenizer = BertWordPieceTokenizer(
#     clean_text=True,
#     handle_chinese_chars=False,
#     strip_accents=False,
#     lowercase=False
# )
# tokenizer.train_from_iterator(X_train, vocab_size=30_000, min_frequency=2,
#                 limit_alphabet=1000, wordpieces_prefix='##',
#                 special_tokens=[
#                     '[PAD', '[UNK]', '[CLS]', '[SEP]', '[MASK]'])
# import os
# os.mkdir('./bert-it')
# tokenizer.save_model('./bert-it', 'bert-it')
# tokenizer = BertTokenizer.from_pretrained('./bert-it/bert-it-vocab.txt')

In [3]:
# Dataset features are "Artist" "Song" "Lyrics"
dataset = load_dataset("chloeliu/lyrics", split='train') # this dataset already removes "the, a ..."" from lyrics

In [4]:
# Exploring dataset
# alphabet = set(' '.join(dataset['train']['lyrics']))
# artists = np.unique(dataset['train']['artist_name'])
# print('billie eilish' in artists)

In [5]:
Y=dataset['artist_name']
print(len(Y))
num_classes = len(np.unique(Y))
print(num_classes)

text = dataset['lyrics']

28372
5426


In [6]:
# the eos and bos tokens are defined
bos = '<|endoftext|>'
eos = '<|EOS|>'
pad = '<|pad|>'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad}

# the new token is added to the tokenizer
num_added_tokens = base_tokenizer.add_special_tokens(special_tokens_dict)

config = AutoConfig.from_pretrained('gpt2', 
                                    bos_token_id=base_tokenizer.bos_token_id,
                                    eos_token_id=base_tokenizer.eos_token_id,
                                    pad_token_id=base_tokenizer.pad_token_id,
                                    output_hidden_states=False)

# the pre-trained model is loaded with the custom configuration
base_model = GPT2LMHeadModel.from_pretrained('gpt2', config=config)

# the model embedding is resized
base_model.resize_token_embeddings(len(base_tokenizer))

Embedding(50259, 768)

In [7]:
new_text = [bos + ' ' + s + ' ' + eos for s in text]

In [8]:
train_split = 0.8
test_split = 0.1

train_ind = int(train_split * (len(dataset)/10))
test_ind = int((1 - test_split) * (len(dataset)/10))

# X_train = (new_text[:train_ind])
# X_valid = (new_text[train_ind:test_ind])
# X_test = (new_text[test_ind:])

Tokenize the text to convert them into numerical inputs that can be fed into the deep learning model. This involves breaking up the text into individual words or subwords, and mapping each token to a unique integer. The tokenization method must be aligned with the chosen model.

In [9]:
def tokenize_function(examples):
        return base_tokenizer(examples['lyrics'], padding=True)

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    num_proc=5,
    remove_columns=list(dataset.features.keys()),
)

In [10]:
base_tokenizer.decode(tokenized_dataset['input_ids'][0])

'hold time feel break feel untrue convince speak voice tear try hold hurt try forgive okay play break string feel heart want feel tell real truth hurt lie worse anymore little turn dust play house ruin run leave save like chase train late late tear try hold hurt try forgive okay play break string feel heart want feel tell real truth hurt lie worse anymore little run leave save like chase train know late late play break string feel heart want feel tell real truth hurt lie worse anymore little know little hold time feel <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pa

In [11]:
# # split tokenized data into training validation and test
# tokenized_X_train = (tokenized_dataset[:train_ind])
# tokenized_X_valid = (tokenized_dataset[train_ind:test_ind])
# tokenized_X_test = (tokenized_dataset[test_ind:])

*   Correct Milestone 1
*   Train Model
*   Evaluate model on test set
*   Update readme with instructions how to run code
*   Add comments to code where needed

In [12]:
from transformers import TrainingArguments, DataCollatorForLanguageModeling, Trainer

model_headlines_path = './output'

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    return {"accuracy": accuracy}

training_args = TrainingArguments(
    output_dir=model_headlines_path,          # output directory
    num_train_epochs=6,              # total # of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=4,   # batch size for evaluation
    warmup_steps=200,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=model_headlines_path,            # directory for storing logs
    prediction_loss_only=True,
    save_steps=10000,
    gradient_accumulation_steps=4
)

data_collator = DataCollatorForLanguageModeling(
        tokenizer=base_tokenizer,
        mlm=False
    )

In [13]:
data_collator([tokenized_dataset[i] for i in range(5)])

{'input_ids': tensor([[ 2946,   640,  1254,  ..., 50258, 50258, 50258],
        [ 6667, 12311,  4268,  ..., 50258, 50258, 50258],
        [34751, 11499,  3758,  ..., 50258, 50258, 50258],
        [41304, 11914,   765,  ..., 50258, 50258, 50258],
        [   83,   359, 40003,  ..., 50258, 50258, 50258]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ 2946,   640,  1254,  ...,  -100,  -100,  -100],
        [ 6667, 12311,  4268,  ...,  -100,  -100,  -100],
        [34751, 11499,  3758,  ...,  -100,  -100,  -100],
        [41304, 11914,   765,  ...,  -100,  -100,  -100],
        [   83,   359, 40003,  ...,  -100,  -100,  -100]])}

In [14]:
train_samples = [tokenized_dataset[i] for i in range(train_ind)]
valid_samples = [tokenized_dataset[i] for i in range(train_ind, test_ind)]
test_samples = [tokenized_dataset[i] for i in range(test_ind, len(tokenized_dataset))]

In [15]:
trainer = Trainer(
    model=base_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    data_collator=data_collator,
    train_dataset=train_samples,         # training dataset
    eval_dataset=valid_samples,
    compute_metrics=compute_metrics         # evaluation dataset
)
trainer.train()
trainer.evaluate()

#OutOfMemoryError: CUDA out of memory. Tried to allocate 216.00 MiB (GPU 0; 14.75 GiB total capacity; 13.61 GiB already allocated; 198.81 MiB free; 13.78 GiB reserved in total by PyTorch) 
#If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


{'eval_loss': 4.527557373046875,
 'eval_runtime': 7.3235,
 'eval_samples_per_second': 38.779,
 'eval_steps_per_second': 9.695,
 'epoch': 6.0}

In [16]:
# this was to calculate how many words appeared more than 10 times

# from collections import defaultdict

# # Store word occurrences
# d = defaultdict(int)
# for phrase in text:
#   for word in phrase.split(' '):
#     d[word] += 1

# # top = (np.argmax(list(d.values())))
# # print(list(d.keys())[top])
# # print(list(d.values())[top])

# # Calculate number of words that appear 10 or more times
# num_words = 0
# for val in list(d.values()):
#   if val >= 10: 
#     num_words += 1
# print(num_words)

In [17]:
# # TODO: change to BERT Tokenization Method

# # Update: limit wordindex to only words with 10 or more appearances
# tokenizer = Tokenizer(num_words=num_words, oov_token="<OOV>")
# # Update: only the training data is used for the calculation of word indices
# tokenizer.fit_on_texts(X_train)
# seq = tokenizer.texts_to_sequences(text)
# print(seq[0])
# Xmat = tokenizer.sequences_to_matrix(seq, mode="tfidf")
# print(Xmat[0])
# print(len(Xmat) == len(X_train))

In [18]:
# output = bert_tokenizer.tokenize(' '.join(X_train))

In [19]:
# Familiarizing with gpt2 model and generating text

# text = "Replace me by any text you'd like."
# outputs = base_model.generate(**base_tokenizer(text, return_tensors="tf"))

# texts = [text, 'ち', 'I love Star Wars']
# encoded_input = [base_tokenizer.encode(text, return_tensors='tf') for text in texts]
# outputs = [base_model.generate(e) for e in encoded_input]

# for output in outputs:
#   print(base_tokenizer.decode(output)) #output[0]
#   print('----------')

In [20]:
# Not sure where labels fit in above

# Y_ind = range(num_classes)
# d = dict(zip(np.unique(Y), Y_ind))
# for i in range(len(Y)):
#   Y[i] = d[Y[i]]
  
# Y_train = np.array(Y[:train_ind])
# Y_valid = np.array(Y[train_ind:test_ind])
# Y_test = np.array(Y[test_ind:])

# print(X_train.shape) # input shape (num_words,)
# Y_train = to_categorical(Y_train,num_classes=num_classes)
# Y_valid = to_categorical(Y_valid,num_classes=num_classes)
# Y_test = to_categorical(Y_test,num_classes=num_classes)

# Y_test[:5]

# X_train = text[:train_ind]
# X_train = np.array(Xmat[:train_ind])
# X_valid = np.array(Xmat[train_ind:test_ind])
# X_test = np.array(Xmat[test_ind:])

In [21]:
# should we standardize (mean/std)
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train = (X_train - mean) / std
X_valid = (X_valid - mean) / std
X_test = (X_test - mean) / std

# RuntimeWarning: invalid value encountered in true_divide

NameError: ignored

Ignore: stop words nltk testing

In [ ]:
# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# # from nltk.stem import PorterStemmer
# nltk.download('stopwords')
# nltk.download('punkt')

In [ ]:
# example_sent = """This is a sample sentence,
#                   showing off the stop words filtration."""
  
# stop_words = set(stopwords.words('english'))
# # ps = PorterStemmer()
# # use with ps.stem(str)

# word_tokens = word_tokenize(example_sent)
# filtered_sentence = [w for w in word_tokens if w not in stop_words]
# print(word_tokens)
# print(filtered_sentence)